In [1]:
import pandas as pd
from datetime import datetime, timedelta
pd.options.display.max_columns = 99

In [2]:
customer = pd.read_csv('./data/Customer.csv')
isin = pd.read_csv('./data/Isin.csv')
market = pd.read_csv('./data/Market.csv')
marketMacro = pd.read_csv('./data/MarketData_Macro.csv')
trade = pd.read_csv('./data/Trade.csv')
challenge = pd.read_csv('./data/Challenge_20180423.csv')
All = [customer, isin, market, marketMacro, trade, challenge]

In [3]:
for item in All:
    print (item.columns)
    print (item.shape)
    print ("\n")

Index(['CustomerIdx', 'Sector', 'Subsector', 'Region', 'Country'], dtype='object')
(3471, 5)


Index(['IsinIdx', 'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey',
       'Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'IssuedAmount', 'CouponType'],
      dtype='object')
(27411, 17)


Index(['IsinIdx', 'DateKey', 'Price', 'Yield', 'ZSpread'], dtype='object')
(9867747, 5)


Index(['DateKey', 'SSE', 'DAX', 'EUROSTOXX', 'VSTOXX', 'FTSE100', 'HSI',
       'NIKKEI', 'DOWJONES_INDU', 'SP500',
       ...
       'Swap_TRY2Y', 'Swap_TRY5Y', 'Swap_USD10Y', 'Swap_USD2Y', 'Swap_USD30Y',
       'Swap_USD5Y', 'Swap_ZAR10Y', 'Swap_ZAR2Y', 'Swap_ZAR30Y', 'Swap_ZAR5Y'],
      dtype='object', length=112)
(877, 112)


Index(['TradeDateKey', 'CustomerIdx', 'IsinIdx', 'BuySell', 'NotionalEUR',
       'Price', 'TradeStatus', 'CustomerInterest'],
      dtype='object')
(6762021, 8)


In [3]:
def convertDatetime(x):
    x = str(x)
    date = x[:4] + '-' + x[4:6] + '-' + x[6:]
    return datetime.strptime(date, '%Y-%m-%d')

In [4]:
trade['date'] = trade['TradeDateKey'].apply(lambda x: convertDatetime(x))

In [5]:
trade['week'] = trade['date'].apply(lambda x: x - timedelta(days=x.weekday()))

In [6]:
trade = trade.drop(['TradeStatus', 'TradeDateKey', 'NotionalEUR', 'date', 'Price'], axis=1)

In [7]:
tradebyweek = trade.groupby(by=['week', 'CustomerIdx', 'IsinIdx', 'BuySell'])['CustomerInterest'].mean().reset_index()

In [8]:
tradebyweek['CustomerInterest'] = tradebyweek['CustomerInterest'].apply(lambda x: 1. if x > 0 else 0.)

In [9]:
# 1) join with isinidx
byweek = tradebyweek.merge(isin, on = 'IsinIdx', how='left')

In [10]:
# 2) join with customer
byweek = byweek.merge(customer, on='CustomerIdx', how='left')

In [11]:
market.head()

,IsinIdx,DateKey,Price,Yield,ZSpread
0,1,20160101,104.250,7.835,5.505
1,7,20160101,107.500,7.520,5.541
2,102,20160101,100.746,4.048,2.085
3,331,20160101,112.790,-0.752,-0.215
4,345,20160101,113.383,-0.667,-0.272


In [12]:
market['date'] = market['DateKey'].apply(lambda x: convertDatetime(x))

In [13]:
market['week'] = market['date'].apply(lambda x: x - timedelta(days=x.weekday()))

In [14]:
marketbyweek = market.drop(['DateKey','date'], axis=1)

In [15]:
marketgrouped = marketbyweek.groupby(by=['week','IsinIdx'])['Price','Yield','ZSpread'].mean().reset_index()

In [16]:
# convert week column: - 7 days for lag
marketgrouped['week'] = marketgrouped['week'].apply(lambda x: x + timedelta(days=7))

In [17]:
byweek.head()

,week,CustomerIdx,IsinIdx,BuySell,CustomerInterest,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region_x,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType,Sector,Subsector,Region_y,Country
0,2015-12-28,10,312,Buy,0.0,3197,20280301,20171213,SEN,USD,FLOW G10,AMERICAS,CDS AND HY,CDS AND HY,US HY YANKEE,BB-,Communications,Cable/Satellite TV,Priv placement,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA
1,2015-12-28,10,316,Sell,0.0,3099,20201125,20131125,SEN,CAD,FLOW G10,AMERICAS,CDS AND HY,CDS AND HY,US HY FIN AUTOS,BBB+,Communications,Telephone-Integrated,Global,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA
2,2015-12-28,10,2686,Sell,0.0,722,20170619,20070619,SEN,USD,SAS & COVERED BONDS,USD SAS,USD SAS,USD SAS,USD SAS,AAA,Financial,Mortgage Banks,Euro mtn,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA
3,2015-12-28,10,4305,Buy,0.0,24,20250728,20150728,SUB,USD,FLOW G10,AMERICAS,HG CASH,HG CASH FIN,YANKEES FIN LT2,BBB,Financial,Commer Banks Non-US,Priv placement,1.500000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA
4,2015-12-28,10,4306,Sell,0.0,24,20170202,20120202,SEN,USD,FLOW G10,AMERICAS,HG CASH,HG CASH FIN,YANKEES FIN SHORT CASH,A,Financial,Money Center Banks,Priv placement,1.500000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA


In [18]:
# 3) join with market
combined = byweek.merge(marketgrouped,on =['week','IsinIdx'], how='left')

In [19]:
combined.head()

,week,CustomerIdx,IsinIdx,BuySell,CustomerInterest,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region_x,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType,Sector,Subsector,Region_y,Country,Price,Yield,ZSpread
0,2015-12-28,10,312,Buy,0.0,3197,20280301,20171213,SEN,USD,FLOW G10,AMERICAS,CDS AND HY,CDS AND HY,US HY YANKEE,BB-,Communications,Cable/Satellite TV,Priv placement,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN
1,2015-12-28,10,316,Sell,0.0,3099,20201125,20131125,SEN,CAD,FLOW G10,AMERICAS,CDS AND HY,CDS AND HY,US HY FIN AUTOS,BBB+,Communications,Telephone-Integrated,Global,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN
2,2015-12-28,10,2686,Sell,0.0,722,20170619,20070619,SEN,USD,SAS & COVERED BONDS,USD SAS,USD SAS,USD SAS,USD SAS,AAA,Financial,Mortgage Banks,Euro mtn,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN
3,2015-12-28,10,4305,Buy,0.0,24,20250728,20150728,SUB,USD,FLOW G10,AMERICAS,HG CASH,HG CASH FIN,YANKEES FIN LT2,BBB,Financial,Commer Banks Non-US,Priv placement,1.500000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN
4,2015-12-28,10,4306,Sell,0.0,24,20170202,20120202,SEN,USD,FLOW G10,AMERICAS,HG CASH,HG CASH FIN,YANKEES FIN SHORT CASH,A,Financial,Money Center Banks,Priv placement,1.500000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN


In [20]:
len(combined)

6262730

In [21]:
combined.columns

Index(['week', 'CustomerIdx', 'IsinIdx', 'BuySell', 'CustomerInterest',
       'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey', 'Seniority',
       'Currency', 'ActivityGroup', 'Region_x', 'Activity', 'RiskCaptain',
       'Owner', 'CompositeRating', 'IndustrySector', 'IndustrySubgroup',
       'MarketIssue', 'IssuedAmount', 'CouponType', 'Sector', 'Subsector',
       'Region_y', 'Country', 'Price', 'Yield', 'ZSpread'],
      dtype='object')

In [22]:
combined.to_csv('./data/byweek_lag.csv', index=False)